{ “cells”: \[ { “cell_type”: “markdown”, “metadata”: {}, “source”: \[
“\# ORE-powered trade risk demo (Indian equities/options)”, “”, “This
notebook demonstrates a lean, explainable risk workflow for pre-trade
sizing, portfolio VaR/ES, and stress testing using a placeholder ORE
analytics stub. Replace the stub with your real ORE bindings to go
live.”, “”, “Contents:”, “- Mock market data generation (OHLCV, option
IVs)”, “- Pre-trade risk checks and position sizing”, “- Portfolio
VaR/ES and stress scenarios”, “- Visualizations for investor demo”, “-
Export endpoints preview (without running a server inside the notebook)”
\] }, { “cell_type”: “code”, “execution_count”: null, “metadata”: {},
“outputs”: \[\], “source”: \[ “#!pip install pandas numpy matplotlib
seaborn pydantic”, “import pandas as pd”, “import numpy as np”, “import
matplotlib.pyplot as plt”, “import seaborn as sns”, “from dataclasses
import dataclass”, “from typing import Dict, List”,
“sns.set(style="whitegrid")”, “”, “np.random.seed(42)” \] }, {
“cell_type”: “markdown”, “metadata”: {}, “source”: \[ “\## ORE analytics
stub (replace with real ORE bindings)”, “”, “This stub mimics key
outputs: price, Greeks, VaR/ES, stress. Swap with ORE SWIG/REST in
production.” \] }, { “cell_type”: “code”, “execution_count”: null,
“metadata”: {}, “outputs”: \[\], “source”: \[ “class OREAnalyticsStub:”,
” def price(self, instrument: Dict, market: Dict):“,” \# Return mock
pricing and sensitivities“,” base_price = market.get("price", 100.0)“,”
return {“,” "npv": float(base_price),“,” "delta": 0.62 if
instrument.get("type") == "Equity" else 0.45,“,” "gamma": 0.02,“,”
"vega": 0.80“,” }“,”“,” def portfolio_var(self, positions: List\[Dict\],
market: Dict, horizon_days: int = 1, method: str = "historical"):“,” \#
Mock VaR/ES increasing with total notional“,” total_notional =
sum(p.get("notional", 0.0) for p in positions)“,” equity =
market.get("equity", 100000.0)“,” var = min(0.03, 0.01 + 0.00000005 \*
total_notional) \# cap at 3%“,” es = var \* 1.4“,” return {"VaR": var,
"ES": es}“,”“,” def stress(self, positions: List\[Dict\], market: Dict,
shocks: Dict\[str, float\]):“,” \# Simple linear shock model“,”
shock_results = {}“,” for name, magnitude in shocks.items():“,” pnl =
0.0“,” for p in positions:“,” delta = p.get("delta", 0.6)“,” price =
p.get("price", 100.0)“,” qty = p.get("qty", 0)“,” pnl += qty \* price \*
delta \* magnitude“,” shock_results\[name\] = pnl /
max(market.get("equity", 1.0), 1.0) \# as fraction of equity“,” return
shock_results“,”“,”ORE = OREAnalyticsStub()” \] }, { “cell_type”:
“markdown”, “metadata”: {}, “source”: \[ “\## Mock market data (INFY
example)”, “We simulate 90 days of OHLCV and a simple IV surface
snapshot.” \] }, { “cell_type”: “code”, “execution_count”: null,
“metadata”: {}, “outputs”: \[\], “source”: \[ “dates =
pd.date_range(end=pd.Timestamp.today().normalize(), periods=90,
freq="B")”, “price = 1500 + np.cumsum(np.random.normal(0, 5,
len(dates)))”, “high = price + np.random.uniform(5, 15, len(dates))”,
“low = price - np.random.uniform(5, 15, len(dates))”, “open\_ = price +
np.random.normal(0, 3, len(dates))”, “close = price +
np.random.normal(0, 3, len(dates))”, “volume = np.random.randint(5e6,
12e6, len(dates))”, “”, “ohlcv = pd.DataFrame({”, ” "timestamp":
dates,“,” "open": open\_,“,” "high": high,“,” "low": low,“,” "close":
close,“,” "volume": volume“,”})“,”ohlcv\["symbol"\] =
"INFY"“,”ohlcv.tail()” \] }, { “cell_type”: “markdown”, “metadata”: {},
“source”: \[ “Option chain (simplified IVs across strikes and one
expiry):” \] }, { “cell_type”: “code”, “execution_count”: null,
“metadata”: {}, “outputs”: \[\], “source”: \[ “exp = (dates\[-1\] +
pd.Timedelta(days=21)).date()”, “strikes = np.arange(1400, 1601, 20)”,
“ivs = 0.20 + 0.05 \* (strikes - 1500) / 1000 \# smile-ish”,
“option_chain = pd.DataFrame({”, ” "symbol": \["INFY"\] \*
len(strikes),“,” "expiry": \[exp\] \* len(strikes),“,” "strike":
strikes,“,” "type": \["Call"\] \* len(strikes),“,” "timestamp":
\[dates\[-1\]\] \* len(strikes),“,” "bid": np.maximum(0.5,
np.random.uniform(5, 25, len(strikes))),“,” "ask": np.random.uniform(25,
45, len(strikes)),“,” "iv": ivs,“,” "oi": np.random.randint(10000,
100000, len(strikes))“,”})“,”option_chain.head()” \] }, { “cell_type”:
“markdown”, “metadata”: {}, “source”: \[ “\## Utility functions: ATR,
bands, sizing”, “These are simple and explainable; tune with your regime
logic later.” \] }, { “cell_type”: “code”, “execution_count”: null,
“metadata”: {}, “outputs”: \[\], “source”: \[ “def compute_atr(df:
pd.DataFrame, period: int = 14) -\> float:”, ” tr = pd.concat(\[“,”
(df\["high"\] - df\["low"\]).abs(),“,” (df\["high"\] -
df\["close"\].shift(1)).abs(),“,” (df\["low"\] -
df\["close"\].shift(1)).abs()“,” \], axis=1).max(axis=1)“,” atr =
tr.rolling(period).mean().iloc\[-1\]“,” return float(atr)“,”“,”def
entry_band(price: float, width_bps: int = 50):“,” w = price \* width_bps
/ 10000.0“,” return \[round(price - w, 2), round(price + w,
2)\]“,”“,”def target_band(price: float, low_pct: float = 0.05, high_pct:
float = 0.08):“,” return \[round(price \* (1 + low_pct), 2), round(price
\* (1 + high_pct), 2)\]“,”“,”def size_by_var(account_equity: float,
current_var: float, var_cap: float, delta: float, price: float,
assumed_move: float = 0.015):“,” max_inc_var = max(var_cap -
current_var, 0.0)“,” if max_inc_var \<= 0:“,” return 0“,”
rupee_var_per_share = delta \* price \* assumed_move / account_equity“,”
size = int(max_inc_var / max(rupee_var_per_share, 1e-9))“,” return
max(size, 0)” \] }, { “cell_type”: “markdown”, “metadata”: {}, “source”:
\[ “\## Pre-trade risk check for INFY”, “We compute ATR stop,
entry/target bands, and size under a user-specific VaR cap.” \] }, {
“cell_type”: “code”, “execution_count”: null, “metadata”: {}, “outputs”:
\[\], “source”: \[ “latest_row = ohlcv.iloc\[-1\]”, “spot =
float(latest_row\["close"\]) ”, “atr = compute_atr(ohlcv)”, “market =
{"price": spot, "equity": 500000.0}”, “”, “analytics =
ORE.price({"type": "Equity", "symbol": "INFY"}, market)”,
“portfolio_metrics = ORE.portfolio_var(\[\], market, horizon_days=1)”,
“”, “user_var_cap = 0.02 \# 2% of equity (Moderate tier)”, “size =
size_by_var(market\["equity"\], portfolio_metrics\["VaR"\],
user_var_cap, analytics\["delta"\], spot)”, “”, “pre_trade_card = {”, ”
"stock": "INFY",“,” "price": round(spot, 2),“,” "entry_price_band":
entry_band(spot, width_bps=50),“,” "stop_loss": round(spot - 1.2 \* atr,
2),“,” "target_price_band_21D": target_band(spot, 0.05, 0.08),“,”
"suggested_size_shares": int(size),“,” "current_portfolio_var":
round(portfolio_metrics\["VaR"\], 4),“,” "post_trade_var_estimate":
round(min(user_var_cap, portfolio_metrics\["VaR"\] + 0.002), 4),“,”
"greeks": {"delta": analytics\["delta"\], "gamma": analytics\["gamma"\],
"vega": analytics\["vega"\]},“,” "notes": \[“,” "Sizing keeps 1-day VaR
under user cap",“,” "ATR-based stop for disciplined exit",“,” "Targets
aligned to \~21D horizon"“,” \]“,”}“,”pre_trade_card” \] }, {
“cell_type”: “markdown”, “metadata”: {}, “source”: \[ “\##
Visualizations: price with ATR bands, and VaR gauge”, “These plots help
investors/users understand risk and rationale.” \] }, { “cell_type”:
“code”, “execution_count”: null, “metadata”: {}, “outputs”: \[\],
“source”: \[ “fig, ax = plt.subplots(2, 1, figsize=(10, 8))”, “”, “\#
Price and ATR bands”, “ax\[0\].plot(ohlcv\["timestamp"\],
ohlcv\["close"\], label="Close", color="#2a9d8f")”,
“ax\[0\].axhline(pre_trade_card\["stop_loss"\], color="#e76f51",
linestyle="–", label="Stop")”,
“ax\[0\].axhline(pre_trade_card\["target_price_band_21D"\]\[0\],
color="#457b9d", linestyle=":", label="Target Low")”,
“ax\[0\].axhline(pre_trade_card\["target_price_band_21D"\]\[1\],
color="#1d3557", linestyle=":", label="Target High")”,
“ax\[0\].set_title("INFY price with stop & target bands")”,
“ax\[0\].legend()”, “”, “\# VaR gauge-like bar”, “var_val =
pre_trade_card\["current_portfolio_var"\]”, “cap_val = user_var_cap”,
“ax\[1\].bar(\["Current VaR", "User Cap"\], \[var_val, cap_val\],
color=\["#f4a261", "#264653"\]) ”, “ax\[1\].set_ylim(0, max(0.05,
cap_val + 0.01))”, “ax\[1\].set_title("Portfolio VaR vs. user cap
(fraction of equity)")”, “for i, v in enumerate(\[var_val, cap_val\]):”,
” ax\[1\].text(i, v + 0.002, f"{v:.3f}",
ha=‘center’)“,”“,”plt.tight_layout()“,”plt.show()” \] }, { “cell_type”:
“markdown”, “metadata”: {}, “source”: \[ “\## Portfolio VaR/ES and
stress scenarios”, “We build a simple portfolio and apply spot and
volatility shocks.” \] }, { “cell_type”: “code”, “execution_count”:
null, “metadata”: {}, “outputs”: \[\], “source”: \[ “positions = \[”, ”
{"symbol": "INFY", "qty": 50, "price": spot, "delta":
analytics\["delta"\], "notional": spot \* 50},”, ” {"symbol":
"HDFCBANK", "qty": 30, "price": spot \* 0.9, "delta": 0.58, "notional":
spot \* 0.9 \* 30}”, ”\]”, “market_ctx = {"equity": 500000.0, "price":
spot}”, “var_es = ORE.portfolio_var(positions, market_ctx)”, “stresses =
ORE.stress(positions, market_ctx, shocks={"spot\_-5pct": -0.05,
"vol_spike_20pct": 0.20})”, “”, “risk_report = {”, ” "VaR":
round(var_es\["VaR"\], 4),“,” "ES": round(var_es\["ES"\], 4),“,”
"stress": {k: round(v, 4) for k, v in
stresses.items()}“,”}“,”risk_report” \] }, { “cell_type”: “markdown”,
“metadata”: {}, “source”: \[ “\### Visualize stress impacts (as % of
equity)” \] }, { “cell_type”: “code”, “execution_count”: null,
“metadata”: {}, “outputs”: \[\], “source”: \[ “labels =
list(risk_report\["stress"\].keys())”, “vals =
list(risk_report\["stress"\].values())”, “plt.figure(figsize=(8,4))”,
“sns.barplot(x=labels, y=vals, palette=\["#e76f51", "#457b9d"\]) ”,
“plt.title("Stress scenario impact (fraction of equity)")”,
“plt.ylabel("Impact")”, “for i, v in enumerate(vals):”, ” plt.text(i,
v + 0.001, f"{v:.3f}", ha=‘center’)“,”plt.show()” \] }, { “cell_type”:
“markdown”, “metadata”: {}, “source”: \[ “\## Recommendation card
(JSON)”, “This is what you can return from your backend to the app UI.”
\] }, { “cell_type”: “code”, “execution_count”: null, “metadata”: {},
“outputs”: \[\], “source”: \[ “import json”,
“print(json.dumps(pre_trade_card, indent=2))” \] }, { “cell_type”:
“markdown”, “metadata”: {}, “source”: \[ “\## Endpoint preview (without
running a server here)”, “Example payload/response shapes for a FastAPI
backend integrating these functions.” \] }, { “cell_type”: “code”,
“execution_count”: null, “metadata”: {}, “outputs”: \[\], “source”: \[
“from pydantic import BaseModel”, “”, “class
PreTradeRequest(BaseModel):”, ” symbol: str“,” timestamp: str“,”
account_equity: float“,” var_cap: float“,”“,”class
PreTradeResponse(BaseModel):“,” stock: str“,” price: float“,”
entry_price_band: List\[float\]“,” stop_loss: float“,”
target_price_band_21D: List\[float\]“,” suggested_size_shares: int“,”
current_portfolio_var: float“,” post_trade_var_estimate: float“,”
greeks: Dict\[str, float\]“,” notes: List\[str\]“,”“,”\# Example of
validating our card against the response model“,”resp =
PreTradeResponse(“,” stock=pre_trade_card\["stock"\],“,”
price=pre_trade_card\["price"\],“,”
entry_price_band=pre_trade_card\["entry_price_band"\],“,”
stop_loss=pre_trade_card\["stop_loss"\],“,”
target_price_band_21D=pre_trade_card\["target_price_band_21D"\],“,”
suggested_size_shares=pre_trade_card\["suggested_size_shares"\],“,”
current_portfolio_var=pre_trade_card\["current_portfolio_var"\],“,”
post_trade_var_estimate=pre_trade_card\["post_trade_var_estimate"\],“,”
greeks=pre_trade_card\["greeks"\],“,”
notes=pre_trade_card\["notes"\]“,”)“,”resp.dict()” \] }, { “cell_type”:
“markdown”, “metadata”: {}, “source”: \[ “\## Next steps to go live with
ORE”, “- Replace `OREAnalyticsStub` with real ORE bindings (SWIG Python
or your REST wrapper)”, “- Feed real NSE/BSE quotes and option chains
from your broker APIs”, “- Extend stress scenarios for earnings days,
sector shocks, volatility regimes”, “- Add user tier mapping:
Conservative (1%), Moderate (2%), Aggressive (3%) VaR caps”, “- Log
every recommendation (inputs/features/model/ORE version) for audit and
compliance” \] } \], “metadata”: { “kernelspec”: { “display_name”:
“Python 3”, “language”: “python”, “name”: “python3” }, “language_info”:
{ “codemirror_mode”: { “name”: “ipython”, “version”: 3 },
“file_extension”: “.py”, “mimetype”: “text/x-python”, “name”: “python”,
“nbconvert_exporter”: “python”, “pygments_lexer”: “ipython3”, “version”:
“3.10” } }, “nbformat”: 4, “nbformat_minor”: 5 }